In [1]:
import nltk
import numpy as np
import pandas as pd
import codecs
import json
import glob

In [2]:
#exported frame from Database

doc = codecs.open('../../../data/classifiedContent.csv','rU','UTF-16') #open for reading with "universal" type set
df = pd.read_csv(doc, sep='\t')

#fix unicode umlaute
# Define a function to replace the Unicode character codes with their corresponding characters
def replace_unicode(s):
    return s.encode('utf-8').decode('unicode_escape')

# Apply the function to the 'country' column of the dataframe
df['rtrJSON'] = df['rtrJSON'].apply(replace_unicode)

print(df.dtypes)
df.head()

ID_GodotObject     int64
rtrJSON           object
dtype: object


,ID_GodotObject,rtrJSON
0,2000120059197,"[{""s"":""keyword"",""k"":""Coronavirus"",""v"":155.4460..."
1,2000120089418,"[{""s"":""event"",""k"":""Corona-Pandemie"",""v"":20.713..."
2,2000122402488,"[{""s"":""keyword"",""k"":""Maskenpflicht"",""v"":169.89..."
3,2000120100164,"[{""s"":""keyword"",""k"":""Sperrstunde"",""v"":93.19406..."
4,2000122440975,"[{""s"":""keyword"",""k"":""Testbeteiligung"",""v"":96.4..."


In [8]:
obj = df['rtrJSON'][10]
obj

'[{"s":"keyword","k":"Neuinfektion","v":98.73378},{"s":"keyword","k":"Arztpraxis","v":84.17109},{"s":"keyword","k":"Coronavirus","v":49.601624},{"s":"keyword","k":"COVID-19","v":14.528863},{"s":"keyword","k":"Corona-Krise","v":10.772496},{"s":"location","k":"Wien","v":85.202034},{"s":"location","k":"Österreich","v":52.750423},{"s":"location","k":"Niederösterreich","v":43.43803},{"s":"location","k":"Tschechien","v":24.533108},{"s":"location","k":"Reichenau an der Rax","v":8.612054},{"s":"location","k":"München","v":6.10781},{"s":"organisation","k":"ÖVP","v":10.055179},{"s":"organisation","k":"ČTK","v":6.205433},{"s":"person","k":"Roman Prymula","v":14.787776},{"s":"person","k":"Karl Nehammer","v":10.288625},{"s":"person","k":"Susanne Raab","v":10.140068},{"s":"person","k":"Irmgard Lechner","v":7.6528015},{"s":"person","k":"Miloš Zeman","v":7.122241},{"s":"person","k":"Andrej Babiš","v":6.320034}]'

In [4]:
def join_tags_easy(s):
    data = json.loads(s)
    result = ','.join([d['k'] for d in data if d['k'] is not None])
    return result


#function for joining words to a list
def join_tags(s, tag_list):
    data = json.loads(s)
    tags = [d['k'] for d in data if d['k'] not in tag_list and d['k'] is not None]
    tag_list = tag_list + tags
    return tag_list

In [5]:
df_join = df.copy()
df_join['tags'] = df_join['rtrJSON'].apply(join_tags_easy)
df_join[['ID_GodotObject','tags']].to_csv('../../data/feature/supervise_classified_content.csv', encoding='utf-8', index=False)
df_join.head()

,ID_GodotObject,rtrJSON,tags
0,2000120059197,"[{""s"":""keyword"",""k"":""Coronavirus"",""v"":155.4460...","Coronavirus,Tagesüberblick,Ampelschaltung,Gast..."
1,2000120089418,"[{""s"":""event"",""k"":""Corona-Pandemie"",""v"":20.713...","Corona-Pandemie,Wiener Ballsaison,Wiener Opern..."
2,2000122402488,"[{""s"":""keyword"",""k"":""Maskenpflicht"",""v"":169.89...","Maskenpflicht,Silvesterfeiern,Tagesüberblick,C..."
3,2000120100164,"[{""s"":""keyword"",""k"":""Sperrstunde"",""v"":93.19406...","Sperrstunde,Maskenpflicht,Regelpaket,COVID-19,..."
4,2000122440975,"[{""s"":""keyword"",""k"":""Testbeteiligung"",""v"":96.4...","Testbeteiligung,Tagesüberblick,Massentest,Anre..."


In [6]:
vals = df['rtrJSON'].values

tags = []
for i in vals:
    tags =join_tags(i, tags)
    
tags[0:10]


['Coronavirus',
 'Tagesüberblick',
 'Ampelschaltung',
 'Gastronomie',
 'Sozialkontakt',
 'Salzburg',
 'Eisenstadt',
 'Österreich',
 'Wien',
 'Dänemark']

In [7]:
# Merge all text per cluster
# Find all CSV files in the current directory
filenames = glob.glob('../../data/feature/clusters/*.csv')

# Create a list to store the dataframes
df_classified_clusters = pd.DataFrame()

# Loop over the CSV files
for filename in filenames:
    # Read the file of a cluster into a dataframe
    df_c = pd.read_csv(filename)
    df_c = pd.merge(df, df_c[['ID_GodotObject','cluster_value']], on ="ID_GodotObject")
    vals = df_c['rtrJSON'].values
    tags = []
    for i in vals:
        tags =join_tags(i, tags)
    
    tags_text = ','.join(tags)
    c = df_c['cluster_value'][0]
    df_classified_clusters = df_classified_clusters.append({'cluster' : c, 'tags' : tags_text}, ignore_index = True)
    
df_classified_clusters.sort_values(by=['cluster'], inplace=True)
df_classified_clusters.reset_index(drop=True, inplace=True)
df_classified_clusters.to_csv('../../data/feature/supervise_clustered_content.csv', encoding='utf-8', index=False)
df_classified_clusters.head()

C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_37276\3488700007.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_classified_clusters = df_classified_clusters.append({'cluster' : c, 'tags' : tags_text}, ignore_index = True)
C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_37276\3488700007.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_classified_clusters = df_classified_clusters.append({'cluster' : c, 'tags' : tags_text}, ignore_index = True)
C:\Users\e11925939\AppData\Local\Temp\8\ipykernel_37276\3488700007.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_classified_clusters = df_classified_clusters.append({'cluster' : c, 'tags' : tags_text}, ignore_index = True)
C:\Users\e11925939\AppData\Local\T

,cluster,tags
0,0,"Veranstaltungsbegrenzung,Schutzmaßnahme,COVID-..."
1,1,"Corona-Pandemie,Pandemie,Infektionszahl,Eva Sc..."
2,2,"Corona-Pandemie,Wiener Ballsaison,Wiener Opern..."
3,3,"Wiener Ballsaison,Corona-Pandemie,Coronavirus,..."
4,4,"Pandemiebekämpfung,Mückstein,Impfung,Pandemie,..."


In [ ]:
## Evaluation of the tags

# 0 -> Veranstaltungsbegrenzung, Gastronomie, Dienstleistung
# 1 -> Verfassungsgerichtshof, Infektionszahl
# 2 -> Opernball,Reisewarnung Reisen
# 3 -> Intensivstation, Impfung, Herbert Kickl, Schulstart
# 4 -> Skigebiet, Testmöglichkeit
# 5 -> Massentest, Neuinfektion, Mindestabstand, Verordnung
# 6 -> Impfstatus, Schulbeginn, BioNTech,Pfizer
# 7 -> Protestzug, Rechtsextrem, Kundgebung